# Training Review

A set of visual tools to compare different models for the same training dataset.

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import tqdm
import torch

import sys; sys.path.append("../")
import maskflow

root_dir = Path("/home/hadim/.data/Neural_Network/Maskflow/Shapes")
data_dir = root_dir / "Data"
model_dir = root_dir / "Models"
model_dir.mkdir(exist_ok=True)

# Import the configuration associated with this dataset and network.
config = maskflow.config.load_config(root_dir / "config.yaml")

## Inference

In [2]:
model_name = '2018.11.03-14:05:20'
model_path = model_dir / model_name

config['MODEL']['DEVICE'] = "cpu"
config['DATALOADER']['NUM_WORKERS'] = 16
config['TEST']['IMS_PER_BATCH'] = 5
    
model = maskflow.inference.build_model(config, model_path)

2018-11-03 15:37:09,728:INFO:maskrcnn_benchmark.utils.checkpoint: Loading checkpoint from /home/hadim/.data/Neural_Network/Maskflow/Shapes/Models/2018.11.03-14:05:20/model_0000198.pth


In [3]:
# Number of batch to load
n = 1

# Load some data
data_loader = maskflow.dataset.get_data_loader(config, data_dir, is_train=False)
some_data = [iter(data_loader).next() for _ in range(n)]

# Retrieve category's names
categories = data_loader.dataset.coco.cats

outputs = []
images = []

for batch_image, batch_target, batch_idx in some_data:
    batch_image = batch_image.to(config['MODEL']['DEVICE'])
    with torch.no_grad():
        output = model(batch_image)
        output = [o.to('cpu') for o in output]
        outputs.extend(output)
        images.extend([im for im in batch_image.tensors])

2018-11-03 15:37:10,142:WARNING:maskrcnn_benchmark.data.build: When using more than one image per GPU you may encounter an out-of-memory (OOM) error if your GPU does not have sufficient memory. If this happens, you can reduce SOLVER.IMS_PER_BATCH (for training) or TEST.IMS_PER_BATCH (for inference). For training, you must also adjust the learning rate and schedule length according to the linear scaling rule. See for example: https://github.com/facebookresearch/Detectron/blob/master/configs/getting_started/tutorial_1gpu_e2e_faster_rcnn_R-50-FPN.yaml#L14


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## Model Evaluation

In [ ]:
model_name = '2018.11.03-14:05:20'
model_path = model_dir / model_name

config['MODEL']['DEVICE'] = "cuda"
config['DATALOADER']['NUM_WORKERS'] = 16
config['TEST']['IMS_PER_BATCH'] = 8
    
results = maskflow.inference.run_evaluation(config, model_path, data_dir)